In [2]:
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from SALib.sample import morris
from SALib.analyze import morris as morris_analyze

In [ ]:
from tqdm import tqdm

# Define the path to your Excel file
excel_path = 'path_to_your_excel_file.xlsx'

# Load the workbook and select the worksheet
wb = load_workbook(excel_path)
ws = wb.active

# Define the problem for the Morris method
problem = {
    'num_vars': 3,
    'names': ['var1', 'var2', 'var3'],
    'bounds': [[0, 10], [0, 10], [0, 10]]
}

# Generate samples using the Morris method
param_values = morris.sample(problem, N=1000, num_levels=4, grid_jump=2)

# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['var1', 'var2', 'var3', 'output'])

# Function to evaluate the model with Excel
def evaluate_model(params):
    # Update the model inputs in Excel
    ws['A1'] = params[0]
    ws['A2'] = params[1]
    ws['A3'] = params[2]
    wb.save(excel_path)

    # Re-open the workbook to ensure changes are saved
    wb_reload = load_workbook(excel_path, data_only=True)
    ws_reload = wb_reload.active

    # Retrieve the model output
    output = ws_reload['B1'].value  # Adjust cell reference accordingly
    return output

# Evaluate the model and collect results
def evaluate_and_collect(param_values):
    outputs = []
    for i in tqdm(range(len(param_values))):
        params = param_values[i]
        output = evaluate_model(params)
        results_df.loc[i] = [params[0], params[1], params[2], output]
        outputs.append(output)
    return np.array(outputs)

# Evaluate the model
Y = evaluate_and_collect(param_values)

# Perform Morris analysis
Si = morris_analyze(problem, param_values, Y, conf_level=0.95)

# Save results to an Excel file
results_df.to_excel('sensitivity_analysis_results.xlsx', index=False)

# Plot the sensitivity analysis for each parameter
import matplotlib.pyplot as plt
import seaborn as sns

fig, axs = plt.subplots(3, 1, figsize=(10, 18))

# Plot for var1
var1_means = results_df.groupby('var1')['output'].mean().reset_index()
sns.lineplot(data=var1_means, x='var1', y='output', ax=axs[0])
axs[0].set_title('Sensitivity Analysis of var1')
axs[0].set_xlabel('var1')
axs[0].set_ylabel('Model Output')

# Plot for var2
var2_means = results_df.groupby('var2')['output'].mean().reset_index()
sns.lineplot(data=var2_means, x='var2', y='output', ax=axs[1])
axs[1].set_title('Sensitivity Analysis of var2')
axs[1].set_xlabel('var2')
axs[1].set_ylabel('Model Output')

# Plot for var3
var3_means = results_df.groupby('var3')['output'].mean().reset_index()
sns.lineplot(data=var3_means, x='var3', y='output', ax=axs[2])
axs[2].set_title('Sensitivity Analysis of var3')
axs[2].set_xlabel('var3')
axs[2].set_ylabel('Model Output')

plt.tight_layout()
plt.show()